# Project: Wrangling and Analyze Data

### Import all relevant libraries

In [ ]:
import numpy as np
import pandas as pd
import requests
import tweepy
import json
from timeit import default_timer as timer
from bs4 import BeautifulSoup

## Data Gathering
In the cell below, we gather **all** three pieces of data for this project and load them in the notebook. **Note:** the methods required to gather each data are different.
1. WeRateDogs Twitter archive data (twitter_archive_enhanced.csv) is downloaded directly

In [ ]:
### Reding the twitter achieve, df_1
df_1 = pd.read_csv('twitter-archive-enhanced.csv')

2. To download the tweet image prediction (image_predictions.tsv) the Requests library is used

In [ ]:
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
###  Read tweet image predictions, df_2
response = requests.get(url)
# check response
response.status_code == 200
# Save TSV to file
with open("image_predictions.tsv", mode='wb') as file:
    file.write(response.content)
    df_2 = pd.read_csv('image_predictions.tsv', sep='\t')

3. The Tweepy library is used to query additional data via the Twitter API (tweet_json.txt)

In [ ]:
# Query Twitter API for each tweet in the Twitter archive and save JSON in a text file
# These are hidden to comply with Twitter's API terms and conditions
consumer_key = 'HIDDEN'
consumer_secret = 'HIDDEN'
access_token = 'HIDDEN'
access_secret = 'HIDDEN'
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

In [ ]:
# Query Twitter's API for JSON data for each tweet ID in the Twitter archive
start = timer()
df_list = []
# Save each tweet's returned JSON as a new line in a .txt file
with open('tweet_json.txt', 'w') as file:
    # This loop will likely take 20-30 minutes to run because of Twitter's rate limit
    for tweet_id in df_1['tweet_id']:
        try:
            tweet = api.get_status(tweet_id, tweet_mode = 'extended')
            json.dump(tweet._json, file)
            file.write('\n')
            print (tweet_id, 'success')
            df_list.append({'tweet_id': tweet_id, 'Status': 'Success'})
        except:
            print (tweet_id, 'Failed')
            df_list.append({'tweet_id': tweet_id, 'Status': 'Failed'})
            pass
tweet_status = pd.DataFrame(df_list, columns = ['tweet_id', 'Status'])
end = timer()
print(end - start)
print(tweet_status.groupby('Status').count())

In [ ]:
# read json file, line by line, to create a pandas DataFrame, df_3
df2_list = []
with open('tweet_json.txt') as file:
    for line in file.readlines():
        json_data = json.loads(line)
        df2_list.append(json_data)
df_3 = pd.DataFrame(df2_list, columns = ['id_str','retweet_count', 'favorite_count'])

## Assessing Data
In this section, we detect and document at least **eight (8) quality issues and two (2) tidiness issue**.
**Both** visual assessment and programmatic assessement was used to assess the data.

In [ ]:
df_1.head(20)

In [ ]:
df_1.tail(20)

In [ ]:
df_1.sample(20)

In [ ]:
df_1.shape

In [ ]:
df_1.columns

In [ ]:
df_1.info()

In [ ]:
df_1.tweet_id.duplicated().sum()

In [ ]:
df_1['source'].unique()

In [ ]:
sum(df_1.duplicated())

In [ ]:
df_1[df_1.rating_denominator != 10]

In [ ]:
df_1[df_1.rating_numerator < 10]

In [ ]:
df_1.name.unique()

In [ ]:
df_2.head(20)

In [ ]:
df_2.tail(20)

In [ ]:
df_2.sample(20)

In [ ]:
df_2.shape

In [ ]:
df_2.columns

In [ ]:
df_2.info()

In [ ]:
df_3.head(20)

In [ ]:
df_3.tail(20)

In [ ]:
df_3.sample(20)

In [ ]:
df_3.shape

In [ ]:
df_3.columns

In [ ]:
df_3.info()

### Quality issues
`twitter achieve` Table

1. Table contians retweets, we only want original ratings.
2. Table contians tweet replies, we only want original ratings.
3. Source column in HTML format
4. Timestamp column is datetime not string
5. No rating in tweet with id 810984652412424192
6. Some tweeets have wrong ratings
7. Some tweets have group ratings i.e tweet contains rating of a group of dogs
8. Tweets with decimal rating not properly extracted
9. Dognames in upper and lower case
10. Table contains columns not useful for analysis
11. Wrong data types for tweet_id,rating_denominator,tweet_source, and dog_stage

`image predictions` Table
1. Wrong data types for tweet_id

### Tidiness issues
`image predictions` Table
1. Extract Dog Breed from image predictions 

`twitter achieve` Table
1. One variable in four columns (Dog Stage)
2. Tweet data in different tables

## Cleaning Data (Quality Issues)
In this section, we clean **all** of the quality issues documented while assessing. 

In [ ]:
# we make copies of original pieces of data
df_1_copy = df_1.copy()
df_2_copy = df_2.copy()
df_3_copy = df_3.copy()

### Issue #1: 
`twitter achieve` Table contians retweets, we only want original ratings.

#### Define: Remove rows with retweet

#### Code

In [ ]:
df_1 = df_1[df_1['retweeted_status_id'].isnull()]

#### Test

In [ ]:
df_1['retweeted_status_id'].value_counts()

### Issue #2: 
`twitter achieve` Table contians tweet replies, we only want original ratings.

#### Define: remove all "tweets" that are "replies".

#### Code

In [ ]:
df_1 = df_1[df_1['in_reply_to_status_id'].isnull()]

#### Test

In [ ]:
df_1['in_reply_to_status_id'].value_counts()

### Issue #3: 
`twitter achieve` Source column in HTML format

#### Define
Use BeautifulSoup to extract content in Source column

#### Code

In [ ]:
df_1['tweet_source'] = df_1.source.apply(lambda x: BeautifulSoup(x).find('a').contents[0])

#### Test

In [ ]:
df_1.tweet_source.unique()

### Issue #4: 
`twitter achieve` Timestamp column is datetime not string

#### Define
change timestamp dtype to datetime

#### Code

In [ ]:
df_1['timestamp'] = pd.to_datetime(df_1.timestamp)

#### Test

In [ ]:
df_1.timestamp.info()

### Issue #5: 
`twitter achieve` No rating in tweet with id 810984652412424192

#### Define
Drop row with tweet id 810984652412424192

#### Code

In [ ]:
df_1 = df_1[df_1.tweet_id != 810984652412424192]

#### Test

In [ ]:
df_1[df_1.tweet_id == 810984652412424192]

### Issue #6: 
`twitter achieve` Some tweeets have wrong ratings

#### Define
input correct rating for tweets with wrong rating

#### Code

In [ ]:
def rating_change(t_id,num,den):
    '''
    this function will be used to change ratings of tweets
    t_id: Tweet id
    num: Rating Numerator value
    den: Rating denominator value
    '''
    df_1.rating_numerator = np.where(df_1.tweet_id == t_id, num, df_1.rating_numerator)
    df_1.rating_denominator = np.where(df_1.tweet_id == t_id, den, df_1.rating_denominator)

In [ ]:
rating_change(666287406224695296, 9, 10)
rating_change(740373189193256964, 14, 10)
rating_change(682962037429899265, 10, 10)
rating_change(722974582966214656, 13, 10)
rating_change(716439118184652801, 11, 10)

#### Test

In [ ]:
df_1[df_1.rating_denominator != 10]

### Issue #7: 
`twitter achieve` Some tweets have group ratings i.e tweet contains rating of a group of dogs

#### Define
Find average rating for tweets with group ratings

#### Code

In [ ]:
df_1.rating_numerator = np.where(df_1.rating_denominator != 10, 
                                 df_1.rating_numerator/(df_1.rating_denominator/10), 
                                 df_1.rating_numerator)

df_1.rating_denominator = np.where(df_1.rating_denominator != 10, 10, 
                                 df_1.rating_denominator)

#### Test

In [ ]:
df_1[df_1.rating_denominator != 10]

### Issue #8: 
`twitter achieve` Tweets with decimal rating not properly extracted

#### Define
Properly extract Tweets with decimal rating

#### Code

In [ ]:
df_1['text_dec'] = df_1.text.str.extract('(\d\.\d+)+[/]10')
df_1[~df_1['text_dec'].isnull()]

In [ ]:
rating_change(883482846933004288, 13.5, 10)
rating_change(786709082849828864, 9.75, 10)
rating_change(778027034220126208, 11.27, 10)
rating_change(680494726643068929, 11.26, 10)

#### Test

In [ ]:
df_1[~df_1['text_dec'].isnull()]

### Issue #9: 
`twitter achieve` Dognames in upper and lower case

#### Define
Convert all dog names to lower case

#### Code

In [ ]:
df_1.name = df_1.name.str.lower()

#### Test

In [ ]:
df_1.name.unique()

### Issue #10: 
`twitter achieve` Table contains columns not useful for analysis

#### Define
Drop all irrelevant columns

#### Code

In [ ]:
rmv_cols = ['in_reply_to_status_id', 'in_reply_to_user_id', 'retweeted_status_id',
            'retweeted_status_user_id', 'retweeted_status_timestamp', 'text_dec','source']

#### Test

In [ ]:
df_1.drop(rmv_cols, axis=1, inplace=True)

### Issue #11: 
`twitter achieve` Wrong data types for tweet_id,rating_denominator,tweet_source, and dog_stage

#### Define
Change to appropriate data types

#### Code

In [ ]:
df_1['tweet_id'] = df_1['tweet_id'].astype(str)
df_1['rating_denominator'] = df_1['rating_denominator'].astype(float)
df_1['tweet_source'] = df_1['tweet_source'].astype('category')

#### Test

In [ ]:
df_1.info()

### Issue #12: 
`image predictions` Wrong data types for tweet_id

#### Define
Change to appropriate data type

#### Code

In [ ]:
df_2['tweet_id'] = df_2['tweet_id'].astype(str)

#### Test

In [ ]:
df_2.info()

## Cleaning Data (Tidiness Issues)
In this section, we clean **all** of the tidiness issues documented while assessing. 

### Issue #1: 
`image predictions` One variable in three prediction columns (Dog Breed)

#### Define
Extract Dog Breed from image predictions

#### Code

In [ ]:
df_2 = df_2[(df_2.p1_dog == True) | (df_2.p2_dog == True) | (df_2.p3_dog == True)]

In [ ]:
df_2a = df_2[(df_2.p1_dog == True)]
df_2a.drop(['p1_dog', 'p2','p2_conf', 'p2_dog', 'p3', 'p3_conf', 'p3_dog'], inplace=True, axis=1)
df_2a.rename(columns={'p1': 'dog_breed', 'p1_conf': 'p_conf'}, inplace = True)

In [ ]:
df_2b = df_2[(df_2.p1_dog == False) & (df_2.p2_dog == True)]
df_2b.drop(['p2_dog', 'p1','p1_conf', 'p1_dog', 'p3', 'p3_conf', 'p3_dog'], inplace=True, axis=1)
df_2b.rename(columns={'p2': 'dog_breed', 'p2_conf': 'p_conf'}, inplace = True)

In [ ]:
df_2c = df_2[(df_2.p1_dog == False) & (df_2.p2_dog == False) & (df_2.p3_dog == True)]
df_2c.drop(['p3_dog', 'p1','p1_conf', 'p1_dog', 'p2', 'p2_conf', 'p2_dog'], inplace=True, axis=1)
df_2c.rename(columns={'p3': 'dog_breed', 'p3_conf': 'p_conf'}, inplace = True)

In [ ]:
df_2 = df_2a.append(df_2b)
df_2 = df_2.append(df_2c)

#### Test

In [ ]:
df_2.head(10)

### Issue #2: 
`twitter achieve` One variable in four columns (Dog Stage)

#### Define
Merge dog stages into one column

#### Code

In [ ]:
df_1.doggo.replace('None', '', inplace=True)
df_1.floofer.replace('None', '', inplace=True)
df_1.pupper.replace('None', '', inplace=True)
df_1.puppo.replace('None', '', inplace=True)

In [ ]:
df_1['dog_stage'] = df_1.doggo + df_1.floofer + df_1.pupper + df_1.puppo

In [ ]:
df_1.dog_stage.replace('doggopupper', 'doggo, pupper', inplace=True)
df_1.dog_stage.replace('doggopuppo', 'doggo, puppo', inplace=True)
df_1.dog_stage.replace('doggofloofer', 'doggo, floofer', inplace=True)
df_1.dog_stage.replace('', np.nan, inplace=True)
df_1['dog_stage'] = df_1['dog_stage'].astype('category')

#### Test

In [ ]:
df_1.dog_stage.value_counts()

### Issue #3: 
`twitter achieve` Tweet data in different tables

#### Define
Merge all data into one table

#### Code

In [ ]:
df_master = df_1.merge(df_2, left_on='tweet_id', right_on='tweet_id', how='left')
df_master = df_master.merge(df_3, left_on='tweet_id', right_on='id_str', how='inner')

#### Test

In [ ]:
df_master.head()

Drop all irrelevant columns

In [ ]:
rmv_cols = ['doggo','floofer','pupper', 'puppo', 'id_str', 'created_at']
df_master.drop(rmv_cols, axis=1, inplace=True)

#### Test

In [ ]:
df_master.head()

## Storing Data
Save gathered, assessed, and cleaned master dataset to a CSV file named "twitter_archive_master.csv".

In [ ]:
df_master.to_csv('twitter_archive_master.csv')

## Analyzing and Visualizing Data
In this section, we produce **three (3) insights and one (1) visualization.** from our master data set

In [ ]:
# Lets take a look at the boxplot of dog rating
df_master.rating_numerator.plot.box()

In [ ]:
# to confirm the number outliers
df_master[df_master.rating_numerator > 20]

In [ ]:
# lets remove the two outliers in order to get a better view of our boxplot
df_master = df_master[df_master.rating_numerator < 20]

In [ ]:
# Lets take a look at the boxplot of dog rating again
df_master.rating_numerator.plot.box()

### Insights:


1. Average Dog rating (numerator):

In [ ]:
np.mean(df_master.rating_numerator)

2. Count of dog breed (top five)

In [ ]:
df_master.dog_breed.value_counts()[0:5]

3. Average dog rating of top 10 retweet_count VS Average dog rating of top 10 favorite_count

In [ ]:
a = np.mean(df_master.nlargest(10, 'retweet_count').rating_numerator)
print('Average dog rating of top 10 retweet_count: ', a)

In [ ]:
b = np.mean(df_master.nlargest(10, 'favorite_count').rating_numerator)
print('Average dog rating of top 10 retweet_count: ', b)

### Visualization

In [ ]:
#lets visualize the distribution of dog rating
df_master.rating_numerator.hist()